<a href="https://colab.research.google.com/github/Shalakagirme/pricing-table/blob/main/Grain_Wheat_Claasif.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

# Path to your dataset
dataset_path = '/content/drive/MyDrive/Grain Dataset/Grain Dataset'

# List the classes (folders) inside the dataset
classes = os.listdir(dataset_path)
print("Classes found:", classes)

# Optional: Check number of images in each class
for cls in classes:
    num_images = len(os.listdir(os.path.join(dataset_path, cls)))
    print(f"{cls}: {num_images} images")




Classes found: ['Basmati', 'jowar', 'Bajra']
Basmati: 4894 images
jowar: 4 images
Bajra: 6 images


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to dataset
dataset_path = '/content/drive/MyDrive/Grain Dataset/Grain Dataset'

# Image size and batch size
img_height, img_width = 224, 224
batch_size = 32

# Create ImageDataGenerator for training with some augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% for validation
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

# Load training data
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)


Found 3925 images belonging to 3 classes.
Found 979 images belonging to 3 classes.


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# Load MobileNetV2 with pre-trained ImageNet weights, without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model
base_model.trainable = False

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
output = Dense(3, activation='softmax')(x)  # 3 classes

# Create the model
model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,422,339 (9.24 MB)

 Trainable params: 164,355 (642.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_grain_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,  # you can increase if needed
    callbacks=[early_stop, checkpoint]
)


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 10s/step - accuracy: 0.9961 - loss: 0.0393
Epoch 1: val_accuracy improved from -inf to 0.99898, saving model to best_grain_model.h5


123/123 ━━━━━━━━━━━━━━━━━━━━ 1511s 12s/step - accuracy: 0.9961 - loss: 0.0391 - val_accuracy: 0.9990 - val_loss: 0.0020
Epoch 2/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.9989 - loss: 0.0032
Epoch 2: val_accuracy improved from 0.99898 to 1.00000, saving model to best_grain_model.h5


123/123 ━━━━━━━━━━━━━━━━━━━━ 199s 2s/step - accuracy: 0.9989 - loss: 0.0032 - val_accuracy: 1.0000 - val_loss: 8.8954e-04
Epoch 3/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 8.6071e-04
Epoch 3: val_accuracy did not improve from 1.00000
123/123 ━━━━━━━━━━━━━━━━━━━━ 191s 2s/step - accuracy: 1.0000 - loss: 8.5960e-04 - val_accuracy: 1.0000 - val_loss: 0.0011
Epoch 4/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 5.4942e-04
Epoch 4: val_accuracy did not improve from 1.00000
123/123 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 1.0000 - loss: 5.4896e-04 - val_accuracy: 0.9990 - val_loss: 0.0013
Epoch 5/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 2.3662e-04
Epoch 5: val_accuracy did not improve from 1.00000
123/123 ━━━━━━━━━━━━━━━━━━━━ 205s 2s/step - accuracy: 1.0000 - loss: 2.3757e-04 - val_accuracy: 1.0000 - val_loss: 0.0010
Epoch 6/20
123/123 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 1.0000 - loss: 6.3160e-04
Epo

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

# Path to your test image
test_img_path = '/content/drive/MyDrive/Grain Dataset/Grain Dataset/rice2.jpg'  # change this

# Load the image and resize to 224x224
img = image.load_img(test_img_path, target_size=(224, 224))

# Convert to array and normalize
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # add batch dimension
img_array /= 255.0  # scale pixel values to 0-1


In [ ]:
# Predict class probabilities
pred = model.predict(img_array)

# Get the index of the class with highest probability
class_index = np.argmax(pred, axis=1)[0]


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step


In [ ]:

# Get mapping from the training generator
class_labels = train_generator.class_indices
class_labels = {v: k for k, v in class_labels.items()}  # invert mapping

# Print predicted class
print("Predicted class:", class_labels[class_index])


Predicted class: Basmati
